# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233467826328                   -0.50    7.0         
  2   -7.249939741994       -1.78       -1.39    1.0   8.13ms
  3   -7.250992270721       -2.98       -1.84    2.0   9.00ms
  4   -7.250975443861   +   -4.77       -1.88    1.0   7.49ms
  5   -7.251330983042       -3.45       -2.71    1.0   7.77ms
  6   -7.251337742634       -5.17       -3.08    1.0   7.55ms
  7   -7.251338674079       -6.03       -3.59    2.0   9.08ms
  8   -7.251338789676       -6.94       -4.08    2.0   9.47ms
  9   -7.251338797645       -8.10       -4.56    2.0   10.1ms
 10   -7.251338798602       -9.02       -5.23    1.0   8.28ms
 11   -7.251338798692      -10.04       -5.43    3.0   11.3ms
 12   -7.251338798701      -11.09       -5.79    1.0   8.36ms
 13   -7.251338798704      -11.50       -6.17    2.0   9.24ms
 14   -7.251338798704      -12.18       -6.69    1.0   8.12ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.08166851724423425
[ Info: Arnoldi iteration step 2: normres = 0.39624424732517033
[ Info: Arnoldi iteration step 3: normres = 0.997028246827168
[ Info: Arnoldi iteration step 4: normres = 0.33404523473969966
[ Info: Arnoldi iteration step 5: normres = 0.5694765773092634
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (7.05e-02, 7.59e-02, 4.61e-01, 3.18e-01, 1.81e-02)
[ Info: Arnoldi iteration step 6: normres = 0.23360054995581767
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.36e-02, 9.42e-02, 1.73e-01, 9.02e-02, 7.41e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06482957633779367
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.06e-04, 5.42e-03, 9.12e-03, 2.34e-02, 5.18e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12693631445370876
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.21e